In [23]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [2]:
airline_df = pd.read_csv("cleaned_airline_2018.csv")

abe_weather_df = pd.read_csv("abe_cleaned_weather_2018.csv")
atl_weather_df = pd.read_csv("atl_cleaned_weather_2018.csv")

In [3]:
airline_df.head()

,FL_DATE,ORIGIN,DEST,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,DISTANCE,Alaska Airlines,Allegiant Air,American Airlines,Delta Airlines,...,Hawaiian Airlines,JetBlue Airways,Mesa Airline,PSA Airlines,Republic Airways,SkyWest Airlines,Southwest Airlines,Spirit Airlines,United Airlines,Virgin America
0,2018-01-01,EWR,DEN,268.0,250.0,1605.0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,2018-01-01,LAS,SFO,99.0,83.0,414.0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,2018-01-01,SNA,DEN,134.0,126.0,846.0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,2018-01-01,RSW,ORD,190.0,182.0,1120.0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,2018-01-01,ORD,ALB,112.0,106.0,723.0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [4]:
abe_weather_df.head()

,time,precipitation_sum (mm),rain_sum (mm),snowfall_sum (cm),windspeed_10m_max (km/h),windgusts_10m_max (km/h),et0_fao_evapotranspiration (mm)
0,2018-01-01,0.0,0.0,0.00,17.9,43.2,0.81
1,2018-01-02,0.0,0.0,0.00,16.0,38.9,0.89
2,2018-01-03,0.0,0.0,0.00,9.3,22.7,0.85
3,2018-01-04,10.2,0.0,7.28,29.8,67.0,0.74
4,2018-01-05,0.0,0.0,0.00,30.9,67.3,0.92


In [5]:
airline_df[airline_df['ORIGIN'] == 'ABE']['DEST'].value_counts()

ATL    960
DTW    912
CLT    911
SFB    441
ORD    345
PIE    157
PGD    144
MYR     84
FLL     72
PHL     40
Name: DEST, dtype: int64

In [9]:
abe_atl_airline_df = airline_df[(airline_df['ORIGIN'] == 'ABE') & (airline_df['DEST'] == 'ATL')]
abe_atl_airline_df.rename(columns = {'FL_DATE':'time'}, inplace = True)
abe_atl_airline_df.head()

/Users/ianchia/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:5042: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,time,ORIGIN,DEST,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,DISTANCE,Alaska Airlines,Allegiant Air,American Airlines,Delta Airlines,...,Hawaiian Airlines,JetBlue Airways,Mesa Airline,PSA Airlines,Republic Airways,SkyWest Airlines,Southwest Airlines,Spirit Airlines,United Airlines,Virgin America
3936,2018-01-01,ABE,ATL,149.0,170.0,692.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19725,2018-01-02,ABE,ATL,141.0,119.0,692.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19733,2018-01-02,ABE,ATL,149.0,134.0,692.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
40933,2018-01-03,ABE,ATL,139.0,144.0,692.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
54149,2018-01-03,ABE,ATL,140.0,159.0,692.0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [17]:
origin_weather_cols = {
    'precipitation_sum (mm)': 'origin precipitation_sum (mm)', 
    'rain_sum (mm)': 'origin rain_sum (mm)', 
    'snowfall_sum (cm)': 'origin snowfall_sum (cm)',
    'windspeed_10m_max (km/h)': 'origin windspeed_10m_max (km/h)', 
    'windgusts_10m_max (km/h)': 'origin windgusts_10m_max (km/h)',
    'et0_fao_evapotranspiration (mm)': 'origin et0_fao_evapotranspiration (mm)'
}

dest_weather_cols = {
    'precipitation_sum (mm)': 'dest precipitation_sum (mm)', 
    'rain_sum (mm)': 'dest rain_sum (mm)', 
    'snowfall_sum (cm)': 'dest snowfall_sum (cm)',
    'windspeed_10m_max (km/h)': 'dest windspeed_10m_max (km/h)', 
    'windgusts_10m_max (km/h)': 'dest windgusts_10m_max (km/h)',
    'et0_fao_evapotranspiration (mm)': 'dest et0_fao_evapotranspiration (mm)'
}

abe_atl_airline_df = abe_atl_airline_df.join(abe_weather_df.set_index('time'), on='time')
abe_atl_airline_df.rename(columns = origin_weather_cols, inplace = True)

abe_atl_airline_df = abe_atl_airline_df.join(abe_weather_df.set_index('time'), on='time')
abe_atl_airline_df.rename(columns = dest_weather_cols, inplace = True)

abe_atl_airline_df.head()

,time,ORIGIN,DEST,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,DISTANCE,Alaska Airlines,Allegiant Air,American Airlines,Delta Airlines,...,origin snowfall_sum (cm),origin windspeed_10m_max (km/h),origin windgusts_10m_max (km/h),origin et0_fao_evapotranspiration (mm),dest precipitation_sum (mm),dest rain_sum (mm),dest snowfall_sum (cm),dest windspeed_10m_max (km/h),dest windgusts_10m_max (km/h),dest et0_fao_evapotranspiration (mm)
3936,2018-01-01,ABE,ATL,149.0,170.0,692.0,0,0,0,0,...,0.0,17.9,43.2,0.81,0.0,0.0,0.0,17.9,43.2,0.81
19725,2018-01-02,ABE,ATL,141.0,119.0,692.0,0,0,0,0,...,0.0,16.0,38.9,0.89,0.0,0.0,0.0,16.0,38.9,0.89
19733,2018-01-02,ABE,ATL,149.0,134.0,692.0,0,0,0,0,...,0.0,16.0,38.9,0.89,0.0,0.0,0.0,16.0,38.9,0.89
40933,2018-01-03,ABE,ATL,139.0,144.0,692.0,0,0,0,0,...,0.0,9.3,22.7,0.85,0.0,0.0,0.0,9.3,22.7,0.85
54149,2018-01-03,ABE,ATL,140.0,159.0,692.0,0,0,0,1,...,0.0,9.3,22.7,0.85,0.0,0.0,0.0,9.3,22.7,0.85


In [18]:
abe_atl_airline_df.describe()

,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,DISTANCE,Alaska Airlines,Allegiant Air,American Airlines,Delta Airlines,Endeavor Air,Envoy Air,ExpressJet,...,origin snowfall_sum (cm),origin windspeed_10m_max (km/h),origin windgusts_10m_max (km/h),origin et0_fao_evapotranspiration (mm),dest precipitation_sum (mm),dest rain_sum (mm),dest snowfall_sum (cm),dest windspeed_10m_max (km/h),dest windgusts_10m_max (km/h),dest et0_fao_evapotranspiration (mm)
count,960.000000,960.000000,960.0,960.0,960.0,960.0,960.000000,960.000000,960.0,960.000000,...,960.000000,960.000000,960.000000,960.000000,960.000000,960.000000,960.000000,960.000000,960.000000,960.000000
mean,131.592708,128.290625,692.0,0.0,0.0,0.0,0.376042,0.365625,0.0,0.258333,...,0.226042,15.300625,37.238542,2.590219,4.338021,4.021042,0.226042,15.300625,37.238542,2.590219
std,7.018057,15.422378,0.0,0.0,0.0,0.0,0.484643,0.481856,0.0,0.437946,...,1.465332,5.359266,12.854702,1.743023,8.085590,7.841272,1.465332,5.359266,12.854702,1.743023
min,115.000000,103.000000,692.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.000000,6.800000,13.000000,0.230000,0.000000,0.000000,0.000000,6.800000,13.000000,0.230000
25%,126.000000,118.000000,692.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.000000,11.100000,27.400000,1.200000,0.000000,0.000000,0.000000,11.100000,27.400000,1.200000
50%,130.000000,125.500000,692.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.000000,14.350000,35.300000,2.110000,0.400000,0.200000,0.000000,14.350000,35.300000,2.110000
75%,136.000000,134.000000,692.0,0.0,0.0,0.0,1.000000,1.000000,0.0,1.000000,...,0.000000,18.400000,43.900000,3.830000,4.600000,4.000000,0.000000,18.400000,43.900000,3.830000
max,151.000000,236.000000,692.0,0.0,0.0,0.0,1.000000,1.000000,0.0,1.000000,...,14.560000,37.800000,88.600000,7.190000,55.700000,55.700000,14.560000,37.800000,88.600000,7.190000


In [19]:
abe_atl_airline_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 960 entries, 3936 to 7069364
Data columns (total 36 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   time                                    960 non-null    object 
 1   ORIGIN                                  960 non-null    object 
 2   DEST                                    960 non-null    object 
 3   CRS_ELAPSED_TIME                        960 non-null    float64
 4   ACTUAL_ELAPSED_TIME                     960 non-null    float64
 5   DISTANCE                                960 non-null    float64
 6   Alaska Airlines                         960 non-null    int64  
 7   Allegiant Air                           960 non-null    int64  
 8   American Airlines                       960 non-null    int64  
 9   Delta Airlines                          960 non-null    int64  
 10  Endeavor Air                            960 non-null   

In [22]:
without_airline_cols = [
    'time', 'ORIGIN', 'DEST', 'CRS_ELAPSED_TIME', 'ACTUAL_ELAPSED_TIME',
    'DISTANCE', 'origin precipitation_sum (mm)', 'origin rain_sum (mm)', 
    'origin snowfall_sum (cm)', 'origin windspeed_10m_max (km/h)', 
    'origin windgusts_10m_max (km/h)', 'origin et0_fao_evapotranspiration (mm)', 
    'dest precipitation_sum (mm)', 'dest rain_sum (mm)', 'dest snowfall_sum (cm)',
    'dest windspeed_10m_max (km/h)', 'dest windgusts_10m_max (km/h)',
    'dest et0_fao_evapotranspiration (mm)'
]

abe_atl_airline_df[without_airline_cols].head()

,time,ORIGIN,DEST,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,DISTANCE,origin precipitation_sum (mm),origin rain_sum (mm),origin snowfall_sum (cm),origin windspeed_10m_max (km/h),origin windgusts_10m_max (km/h),origin et0_fao_evapotranspiration (mm),dest precipitation_sum (mm),dest rain_sum (mm),dest snowfall_sum (cm),dest windspeed_10m_max (km/h),dest windgusts_10m_max (km/h),dest et0_fao_evapotranspiration (mm)
3936,2018-01-01,ABE,ATL,149.0,170.0,692.0,0.0,0.0,0.0,17.9,43.2,0.81,0.0,0.0,0.0,17.9,43.2,0.81
19725,2018-01-02,ABE,ATL,141.0,119.0,692.0,0.0,0.0,0.0,16.0,38.9,0.89,0.0,0.0,0.0,16.0,38.9,0.89
19733,2018-01-02,ABE,ATL,149.0,134.0,692.0,0.0,0.0,0.0,16.0,38.9,0.89,0.0,0.0,0.0,16.0,38.9,0.89
40933,2018-01-03,ABE,ATL,139.0,144.0,692.0,0.0,0.0,0.0,9.3,22.7,0.85,0.0,0.0,0.0,9.3,22.7,0.85
54149,2018-01-03,ABE,ATL,140.0,159.0,692.0,0.0,0.0,0.0,9.3,22.7,0.85,0.0,0.0,0.0,9.3,22.7,0.85


In [26]:
fig = px.imshow(
    abe_atl_airline_df[without_airline_cols].corr(),
)
fig.show()